In [10]:
import pandas as pd
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
from openai import OpenAI

In [5]:
file = 'Restaurant_reviews.csv'

In [7]:
# Read data 
df = pd.read_csv(file)
print(df.head())

        Restaurant              Reviewer  \
0  Beyond Flavours     Rusha Chakraborty   
1  Beyond Flavours  Anusha Tirumalaneedi   
2  Beyond Flavours       Ashok Shekhawat   
3  Beyond Flavours        Swapnil Sarkar   
4  Beyond Flavours                Dileep   

                                              Review Rating  \
0  The ambience was good, food was quite good . h...      5   
1  Ambience is too good for a pleasant evening. S...      5   
2  A must try.. great food great ambience. Thnx f...      5   
3  Soumen das and Arun was a great guy. Only beca...      5   
4  Food is good.we ordered Kodi drumsticks and ba...      5   

                  Metadata             Time  Pictures    7514  
0   1 Review , 2 Followers  5/25/2019 15:54         0  2447.0  
1  3 Reviews , 2 Followers  5/25/2019 14:20         0     NaN  
2  2 Reviews , 3 Followers  5/24/2019 22:54         0     NaN  
3    1 Review , 1 Follower  5/24/2019 22:11         0     NaN  
4  3 Reviews , 2 Followers  5/24/201

In [12]:
df = df.drop("7514", axis=1)
df = df.drop("Pictures", axis=1)
# Check for NA and null values in each column
na_null_check = df.isnull().sum()
print("Null rows :- ", na_null_check )


Null rows :-  Restaurant     0
Reviewer      38
Review        45
Rating        38
Metadata      38
Time          38
dtype: int64


In [18]:
df = df[df['Review'].notna()]  # Clean missing reviews
data = df.sample(700).to_dict('records')
data

[{'Restaurant': 'Royal Spicy Restaurant',
  'Reviewer': 'Himaraj Hima',
  'Review': "didn't reach expectations.worst taste. thrown into dustbin.",
  'Rating': '1',
  'Metadata': '1 Review',
  'Time': '7/23/2018 20:34'},
 {'Restaurant': 'The Old Madras Baking Company',
  'Reviewer': 'Basavarajeshwari',
  'Review': 'Oh what a lovely place. Thanks to my friend who took me here for breakfast. Both of us were super hungry. We ordered mushroom omelette with butter crossiant and scarmbled egg with butter crossiant. A jalepeno bagel with cream chees. OMG 😲 what amazing they tasted. The best crossiant I have come across so far in India. So soft. So were the scrambles egg and mushroom omlette. The cream cheese was just amazing. We ordered Hazelnut cafe Mocha and Hazelnut capcuinno. They were just good. Thank go hazelnut syrup they used wasnt very sweet.\nI was so in love with smell of bread they are baking took. banana cake , Danish pastry , Chocolate cake and brioche for my frined. Would defini

In [20]:

# Initialize models
encoder = SentenceTransformer('all-MiniLM-L6-v2')
qdrant = QdrantClient(":memory:")

# Create collection for restaurant reviews
qdrant.recreate_collection(
    collection_name="restaurant_reviews",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)

# Upload reviews with embeddings
qdrant.upload_points(
    collection_name="restaurant_reviews",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["Review"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data)
    ]
)


C:\Users\regos\AppData\Local\Temp\ipykernel_19020\307475754.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


In [22]:

# Example search query
user_prompt = "Find a restaurant for a date night"

# Perform semantic search
hits = qdrant.search(
    collection_name="restaurant_reviews",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)

# Prepare search results
search_results = [hit.payload for hit in hits]

# Connect to LLM for response generation
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1",
    api_key="sk-no-key-required"
)

completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are a restaurant concierge. Help users find dining options based on reviews."},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": str(search_results)}
    ]
)

print(completion.choices[0].message.content)

C:\Users\regos\AppData\Local\Temp\ipykernel_19020\415944753.py:5: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant.search(


Based on the reviews, I would recommend "Eat India Company" for a date night. It has a 5-star rating and 21 reviews, with many reviewers praising the food, ambience, and service. One reviewer even mentioned that it's a "must-visit restaurant" and that the price is worth the experience.

The other two options, "The Glass Onion" and "SKYHY", also have good reviews, but they seem to be more geared towards a casual hangout or a night out with friends rather than a romantic date night. "The Glass Onion" has a 4-star rating and 13 reviews, with reviewers praising the food and service, but mentioning that the prices are a bit high. "SKYHY" has a 5-star rating and 1 review, with the reviewer praising the bar tender and the food, but the review is quite brief.

Eat India Company seems to offer a more romantic and intimate atmosphere, with reviewers mentioning the attractive and cozy ambience, and the tasty food. It's also worth noting that the reviewer who visited twice mentioned that it's a gr